In [44]:
#!pip install pandas
#!pip install indian-names
#!pip install random-word
#!pip install psycopg2
#!pip install faker
#!pip install pyarrow
#!pip install sqlalchemy
#!pip install scikit-learn

In [45]:
#imports
import pandas as pd
import indian_names
import random
import psycopg2
from random_word import RandomWords
from datetime import datetime, timedelta
import pyarrow as pa
import pyarrow.parquet as pq
import os
import json
from collections import Counter
from faker import Faker
from sqlalchemy import create_engine
fake = Faker()

In [46]:
# Function to generate weather data for a given city
import pandas as pd
def generate_weather_data(city,start_date, iterations):
    # Temperature ranges and weather conditions for different seasons (in Celsius)
    summer_temp_range = (25, 40)
    summer_conditions = ['sunny', 'partly cloudy']
    summer_wind_speed_range = (5, 15)  # in km/h
    summer_humidity_range = (30, 60)   # in percentage
    
    rainy_temp_range = (20, 30)
    rainy_conditions = ['rainy','partly cloudy','heavy rain','thunderstorm']
    rainy_wind_speed_range = (10, 25)  # in km/h
    rainy_humidity_range = (60, 90)    # in percentage
    
    winter_temp_range = (10, 25)
    winter_conditions = ['clear', 'partly cloudy']
    winter_wind_speed_range = (5, 10)   # in km/h
    winter_humidity_range = (20, 50)    # in percentage
    
    # Start date for generating weather data
    #start_date = datetime(2022, 1, 1)  # Start from January 1st, 2022
    
    # Initialize list to store weather data
    weather_data = []
    
    # Loop for every 10 minutes within two years
    for _ in range(iterations):  # 24 hours * 365 days * 2 years
        # Randomly select season based on month
        month = start_date.month
        if 3 <= month <= 5:
            season = 'summer'
        elif 6 <= month <= 9:
            season = 'rainy'
        else:
            season = 'winter'
        
        # Randomly select weather condition, temperature, wind speed, and humidity based on season
        if season == 'summer':
            temperature = round(random.uniform(*summer_temp_range), 2)
            weather_condition = random.choice(summer_conditions)
            wind_speed = round(random.uniform(*summer_wind_speed_range), 2)
            humidity = round(random.uniform(*summer_humidity_range), 2)
        elif season == 'rainy':
            temperature = round(random.uniform(*rainy_temp_range), 2)
            weather_condition = random.choices(rainy_conditions,weights = [0.4,0.2,0.2,0.2])[0]
            wind_speed = round(random.uniform(*rainy_wind_speed_range), 2)
            humidity = round(random.uniform(*rainy_humidity_range), 2)
        else:
            temperature = round(random.uniform(*winter_temp_range), 2)
            weather_condition = random.choice(winter_conditions)
            wind_speed = round(random.uniform(*winter_wind_speed_range), 2)
            humidity = round(random.uniform(*winter_humidity_range), 2)
        
        # Append weather data to the list
        weather_data.append({
            'city': city,
            'time': start_date,
            'condition': weather_condition,
            'temperature': temperature,
            'wind_speed': wind_speed,
            'humidity': humidity
        })
        
        # Increment time by 10 minutes
        start_date += timedelta(minutes=10)
    
    return weather_data


def training_dataset(start_date, iterations):
    # Initialize an empty DataFrame to store weather data
    weather_df = pd.DataFrame()

    # Cities for which weather data will be generated
    cities = ['Mumbai', 'Delhi', 'Pune', 'Hyderabad', 'Bangalore']

    # Generate weather data for each city and append to DataFrame
    for city in cities:
        city_weather_data = generate_weather_data(city,start_date, iterations)
        weather_df = weather_df._append(city_weather_data, ignore_index=True)
        #weather_df = pd.concat([weather_df,pd.DataFrame([city_weather_data])], ignore_index=True)

    # Print the DataFrame
    #print(weather_df)
    
    return weather_df


In [47]:
import xml.etree.ElementTree as ET
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score




def determine_weather_category(row):
    # Define thresholds for weather conditions
    temperature_threshold = 25  # Example threshold for temperature
    humidity_threshold = 70     # Example threshold for humidity
    wind_speed_threshold = 15    # Example threshold for wind speed
    
    # Check temperature, humidity, and wind speed
    if row['temperature'] > temperature_threshold and row['humidity'] < humidity_threshold and row['wind_speed'] < wind_speed_threshold:
        return 'good'
    elif row['temperature'] < temperature_threshold and row['humidity'] > humidity_threshold and row['wind_speed'] > wind_speed_threshold:
        return 'bad'
    #elif row['condition'] in ['sunny', 'partly cloudy']:
    #    return 'good'
    elif row['condition'] in ['thunderstorm']:
        return 'bad'
    else:
        return 'good'


def data_model(data):
    
    # Create DataFrame
    df = pd.DataFrame(data, columns=['city', 'time', 'condition', 'temperature', 'wind_speed', 'humidity'])

    # Define 'good' or 'bad' weather based on conditions (Modify as per requirement)
    #df['weather_category'] = df['condition'].apply(lambda x: 'good' if x in ['sunny', 'rainy' ,'partly cloudy'] else 'bad')

    # Apply the function to create a new weather category column
    df['weather_category'] = df.apply(determine_weather_category, axis=1)

    print(df['city'].unique())

    # Select features and target variable
    X = df[['temperature', 'wind_speed', 'humidity']]
    y = df['weather_category']

    # Split data into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # Train Random Forest classifier
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(X_train, y_train)

    # Make predictions
    y_pred = clf.predict(X_test)

    # Evaluate model
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy:", accuracy)
    return clf
    
    



In [48]:
start_date = datetime.strptime('2022-01-01', "%Y-%m-%d")
iterations = 24*365*2*6
data = training_dataset(start_date, iterations)
model = data_model(data)



['Mumbai' 'Delhi' 'Pune' 'Hyderabad' 'Bangalore']
Accuracy: 0.9497843734145104


In [49]:
# def write_to_postgress(df):
#     username = 'sv_pgdev'
#     password = 'Zathura@5499'
#     host = 'dev-zathura-edw.postgres.database.azure.com'
#     port = '5432'
#     database = 'Landing'

#     # Create a SQLAlchemy engine
#     engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{database}')

#     # Write DataFrame to PostgreSQL table (append if exists, create if not)
#     df.to_sql('flight_schedules', engine, if_exists='append', index=False)


In [50]:
def write_to_postgress(df,table_name):
    # PostgreSQL connection parameters
    username = 'sv_pgdev'
    password = 'Zathura@5499'
    host = 'dev-zathura-edw.postgres.database.azure.com'
    port = '5432'
    database = 'Landing'
    
    try:
        # Establish a connection to the PostgreSQL server
        conn = psycopg2.connect(
            dbname=database,
            user=username,
            password=password,
            host=host,
            port=port
        )
    
        # Create a cursor object
        cursor = conn.cursor()
    
        # Check if the flight_schedules table exists
        # cursor.execute("""
        #     SELECT EXISTS (
        #         SELECT 1
        #         FROM   information_schema.tables 
        #         WHERE  table_schema = 'public'
        #         AND    table_name = 'flight_schedules'
        #     );
        # """)
        # table_exists = cursor.fetchone()[0]
    
        # if not table_exists:
        #     # Create the flight_schedules table if it doesn't exist
        #     cursor.execute("""
        #         CREATE TABLE flight_schedules (
        #             id SERIAL PRIMARY KEY,
        #             A FLOAT,
        #             B FLOAT,
        #             C FLOAT
        #         );
        #     """)

        column_names_str = ', '.join(df.columns)
        
        # Convert DataFrame to list of tuples
        rows = [tuple(x) for x in df.to_numpy()]

        # Construct the placeholder string
        placeholders = ', '.join(['%s'] * len(df.columns))
        #print(placeholders)
    
        # Construct the INSERT INTO statement with parameterized query
        insert_query = f"INSERT INTO {table_name} VALUES ({placeholders})"
    
        # Execute the INSERT statement with executemany
        cursor.executemany(insert_query, rows)

        # Commit the transaction
        conn.commit()
    
        # Close the cursor and connection
        cursor.close()
        conn.close()
    
    except psycopg2.Error as e:
        print("Error: Could not connect to PostgreSQL server:", e)


In [51]:
def generate_aadhar_number():
    return ' '.join(''.join(map(str,random.sample(range(10), 4))) for x in range(3))

In [52]:
def generate_passport_number():
    alphabets = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    letter = random.choices(alphabets)[0]
    number =letter + str(random.randint(1,9)) + str(random.randint(0,9))\
                + ''.join(map(str,random.sample(range(10),4))) + str(random.randint(1,9)) 
    return number

In [53]:
def generate_phone_number():
    phone_number = ''
    
    #first number should be between 6-9
    phone_number += (str(random.randint(6,9)))
    
    for i in range(1, 10): 
        phone_number += (str(random.randint(0, 9))) 
        
    return phone_number

In [54]:
def generate_email(choices):
    number_choice = [''.join(map(str,random.sample(range(10),x))) for x in range(2,5)]
    domain_choice = ['gmail.com','yahoo.com','hotmail.com']
    return random.choices(choices,weights = [0.5,0.3,0.1,0.1])[0] + '.' + random.choices(choices,weights = [0.3,0.5,0.1,0.1])[0] + random.choice(number_choice) + '@' + random.choice(domain_choice)
    

In [55]:
def random_with_N_digits(n):
    range_start = 10**(n-1)
    range_end = (10**n)-1
    return random.randint(range_start, range_end)

In [56]:
class Passenger:
    def __init__(self):
        gender = random.choice(["male","female"])
        self.first_name = indian_names.get_first_name(gender)
        self.last_name = indian_names.get_last_name()
        self.age = random.randint(5, 80)
        if(self.age > 18):
            self.gender = random.choices([gender,"Others"],weights = [0.95,0.05])[0]
        else:
            self.gender = gender
        if(self.age < 15):
            self.contact_number = ""
            self.email_id = ""
            self.last_name = ""
        else: 
            self.contact_number = generate_phone_number()
            self.email_id = generate_email([self.first_name,self.last_name,RandomWords().get_random_word(),RandomWords().get_random_word()])
        self.aadhar_number = generate_aadhar_number()
        self.passport_number = random.choice([generate_passport_number(),""])

In [57]:
class Airplanes:
    def __init__(self,airplane_type,airplane_name,frequency):
        if(airplane_type == 'Boeing 777X'):
            self.fuel_capacity = 350410 #lbs
            self.seats = 54
            self.classes = {
                            "Economy" : { "seat_type" : "D"   ,"seats": 20 ,"window_seats": 9},
                            "Premium_Economy": { "seat_type" : "C" ,"seats": 14 , "window_seats": 8 } ,
                            "Business" : { "seat_type" : "B"   ,"seats": 12,"window_seats": 6} ,
                            "First_class": { "seat_type" : "A"   ,"seats": 8,"window_seats": 3}
                           }
            self.range = 16190 #km
            self.window_seats = 26
            self.base_price = 5000
        elif(airplane_type == 'Airbus A220'):
            self.fuel_capacity = 21805 
            self.seats = 40
            self.classes = {
                            "Economy" : { "seat_type" : "D"   ,"seats": 25 ,"window_seats": 12  },
                            "Premium_Economy": { "seat_type" : "C"   ,"seats": 15,"window_seats": 7}
                            }
            self.range = 6390 #km
            self.window_seats = 19
            self.base_price = 3000
        elif(airplane_type == 'Airbus A320'):
            self.fuel_capacity = 27200
            self.seats = 45
            self.range = 6150
            self.classes = {
                            "Economy" : { "seat_type" : "D"   ,"seats": 15 ,"window_seats": 6},
                            "Premium_Economy": { "seat_type" : "C" ,"seats": 12 , "window_seats": 5 } ,
                            "Business" : { "seat_type" : "B"   ,"seats": 10,"window_seats": 4} ,
                            "First_class": { "seat_type" : "A"   ,"seats": 8,"window_seats": 2}
                           }
            self.window_seats = 17
            self.base_price = 3500
            
        self.name = airplane_name
        self.airplane_id = airplane_name[:3] + '-' + airplane_type[:3]
        self.type = airplane_type
        self.current_fuel = 0
        self.frequency = frequency
        

In [58]:
class Airport:
    def __init__(self,airport_name):
        self.name = airport_name
        self.code = airport_name[:3].upper()
        self.airplanes = set()
        

In [66]:
class Flight:
    def __init__(self,airplane,source_airport,destination_airport,flight_id,start_time,hour):
        curr_airplane = airplane   #We will pick each plane and check whether it wants to go or no
                
        #current airplane data
        self.id = flight_id
        self.total_seats  = curr_airplane.seats  
        self.flight_code = curr_airplane.airplane_id
        self.classes = curr_airplane.classes
        self.base_price = curr_airplane.base_price
        self.estimated_departure_time = start_time
        self.estimated_arrival_time = start_time + timedelta(hours=hour)
        self.departure_time = random.choice([(start_time + timedelta(minutes= - random.randint(0,3)))
                                             ,(start_time + timedelta(minutes= random.randint(0,15)))])
        self.arrival_time = random.choice([(self.departure_time + timedelta(hours=hour) + timedelta(minutes = -random.randint(0,15))) \
                                           ,(self.departure_time + timedelta(hours=hour) + timedelta(minutes = random.randint(0,15)))])
        self.source = source_airport
        self.destination = destination_airport
        self.passengers = [] 
        self.payments = []
        self.status = None
        self.comments = None
        
        
        input_data = generate_weather_data(self.destination,self.arrival_time + timedelta(minutes = -30),6)
        df = pd.DataFrame(input_data, columns=['city', 'time', 'condition', 'temperature', 'wind_speed', 'humidity'])
        df = df[['temperature', 'wind_speed', 'humidity']]
        
        output_data = model.predict(df)
        counts = Counter(output_data)
        #print(counts)

        # Find the most common value
        most_common_value = counts.most_common(1)[0][0]
        self.time_of_payment = self.departure_time + timedelta(hours= -random.randint(4,150)) + timedelta(minutes = -random.randint(0,55))
        
        if most_common_value == 'bad':
            self.status = 'Cancelled'
            self.departure_time = None
            self.arrival_time =None
        else:
            self.status = 'Landed'
       
        
    def get_flight_data(self):
        return {
            "id" : self.id
            ,"flight_code" : self.flight_code
            ,"source" : self.source
            ,"destination" : self.destination
            ,"estimated_departure_time" : self.estimated_departure_time
            ,"departure_time" : self.departure_time
            ,"estimated_arrival_time" : self.estimated_arrival_time
            ,"arrival_time" : self.arrival_time
            ,"passenger_data" : self.passengers
            ,"total_seats" : self.total_seats  
            ,"status" : self.status
        }
        
        
    def fill_flight(self):
        base_price = self.base_price
        flight_classes = self.classes
        for flight_class in list(flight_classes.keys()):
            total_seats = flight_classes[flight_class]["seats"]
            window_seats = flight_classes[flight_class]["window_seats"]
            seat_type = flight_classes[flight_class]["seat_type"]
            while total_seats > 0:
                boarding = random.choices(["Yes","No"],weights = [0.9,0.1])[0]
                if(boarding):
                        cash = base_price
                        seating_class = 'Economy'
                        if(seat_type == 'A'):
                            cash += base_price*0.4
                            seating_class = 'First-class'
                        elif(seat_type == 'B'):
                            cash += base_price*0.3
                            seating_class = 'Business'
                        elif(seat_type == 'C'):
                            cash += base_price*0.2
                            seating_class = 'Premium-Economy'
    
                        seat_number = seat_type +'-'+ str(total_seats)
                        
                        window = random.choices(["Yes","No"],weights = [0.4,0.6])[0]
                        if(window_seats > 0 and window):
                            seat_number = seat_number + '(W)'
                            cash += base_price * 0.1
                            window_seats -= 1
                        
                        passenger = Passenger()

                        payment_mode = random.choices(['Credit/Debit Card','UPI','Cash'],weights = [0.4,0.59,0.01])[0]
                        
                        passenger_data = {
                            "first_name" : passenger.first_name
                            ,"last_name" : passenger.last_name
                            ,"age" : passenger.age
                            ,"gender" : passenger.gender
                            ,"contact_number" : passenger.contact_number
                            ,"email_id" : passenger.email_id
                            ,"aadhar_number" : passenger.aadhar_number
                            ,"passport_number" : passenger.passport_number
                            ,"seating_class": seating_class
                            ,"seat": seat_number
                            ,"fare" : cash
                            ,"mode_of_payment" : payment_mode

                        }
                        
                            
                        self.passengers.append(passenger_data) 
                        
                        transaction_id = -1
                        if(payment_mode != 'Cash'):
                            transaction_id = random_with_N_digits(10)


                        payment_transaction = {
                            "flight_id" : self.id
                            ,"first_name" : passenger.first_name
                            ,"last_name" : passenger.last_name
                            ,"contact_number" : passenger.contact_number
                            ,"email_id" : passenger.email_id
                            ,"seat": seat_number
                            ,"mode_of_payment" : payment_mode
                            ,"time_of_payment" : self.time_of_payment
                            ,"Amount" : cash 
                            ,"TransactionID" : transaction_id
                        }

                        self.payments.append(payment_transaction) 

                        


                
                total_seats -= 1

                #print(f'Loading...{total_seats}')
        df = pd.DataFrame(self.payments)
        df['time_of_payment'] = df['time_of_payment'].astype(str)
        write_to_postgress(df,'payments')
        
        
    

In [70]:
class Zathura:
    
    #Constructor
    def __init__(self):
        
        #Initialization the harcoded value + data structures
        self.global_flight_id = 103988  
        self.aircraft_models = {
            'Boeing 777X': ['Garuda', 'Vimana', 'Vayu', 'Ananta'],
            'Airbus A220': ['Agni', 'Astra', 'Vajra'],
            'Airbus A320': ['Rudra', 'Marut']
        }
        self.airports = ["Mumbai","Delhi","Pune","Banglore","Hydrebad"]
        self.airport_data = {}
        self.flights_data = []
        self.airplane_data = []
        self.airplane_initials = {
            'Garuda' : {'frequency' : '1357' ,'source' : 'Pune', 'destination' : 'Delhi' , 'timings' :[7,12,17,23], 'index': 0, 'hours' : 2} , 
            'Vimana' : {'frequency' : '1357' ,'source' : 'Mumbai', 'destination' : 'Delhi', 'timings' :[6,12,18] , 'index' : 0,'hours' : 2 } ,
            'Vayu' : {'frequency' : '246' ,'source' : 'Hydrebad', 'destination' : 'Delhi', 'timings' :[9,13,19,23] , 'index' : 0, 'hours' : 3},
            'Ananta' : {'frequency' : '246' ,'source' : 'Delhi', 'destination' : 'Banglore','timings' :[6,11,17] , 'index' : 0,'hours' : 4} ,
            'Agni': {'frequency' : '2345' ,'source' : 'Banglore', 'destination' : 'Pune','timings' :[5,10,15,20] , 'index' : 0,'hours' : 3 },
            'Astra' : {'frequency' : '2345' ,'source' : 'Banglore', 'destination' : 'Mumbai','timings' :[7,12,17,23] , 'index' : 0,'hours' : 3},
            'Vajra' : {'frequency' : '234567' ,'source' : 'Pune', 'destination' : 'Hydrebad','timings' :[6,11,16,21] , 'index' : 0,'hours' : 2} ,
            'Rudra' : {'frequency' : '1234567' ,'source' : 'Mumbai', 'destination' : 'Hydrebad','timings' :[12,17] , 'index' : 0,'hours' : 2},
            'Marut' : {'frequency' : '1234567' ,'source' : 'Mumbai', 'destination' : 'Pune','timings' :[6,12,18] , 'index' : 0,'hours' : 1}
        }
        self.start_date = datetime.strptime('2023-07-21', "%Y-%m-%d") 
        self.end_date = datetime.strptime('2023-12-31', "%Y-%m-%d") 
     
    
    #Setting airport data
    def set_airport_data(self):
        for airport in self.airports:
            self.airport_data[airport] = (Airport(airport)) 
            
    #Setting airplane_data
    def set_airplane_data(self):
        for model in self.aircraft_models:
                for model_name in self.aircraft_models[model]:
                        self.airplane_data.append(Airplanes(model,model_name,self.airplane_initials[model_name]['frequency']))
           
    #Setting airplanes to a initial airport
    def set_inital_airplanes_to_airports(self):
        for airplane in self.airplane_data:
            airport = self.airplane_initials[airplane.name]['source']
            self.airport_data[airport].airplanes.add(airplane)

    def load_initaltables(self):
        #flights - 
        flights_df = pd.DataFrame(self.airplane_initials)
        flights_df = flights_df.transpose() 
        flights_df.drop(columns=['index','hours'], inplace=True)
        flights_df.reset_index(inplace=True)
        flights_df.rename(columns={'index': 'name'}, inplace=True)
        write_to_postgress(flights_df,'flights')

        #airplanes- 
        data = []

        for airplane in self.airplane_data:
            data.append( {
                    "airplaneId" : airplane.airplane_id
                    ,"name" : airplane.name
                    ,"type":airplane.type
                    ,"fuel_capacity" : airplane.fuel_capacity
                    ,"total_seats" : airplane.seats
                    ,"classes" : airplane.classes
                    ,"range" : airplane.range
                }
            )

        airplane_df = pd.DataFrame(data)
        airplane_df['classes'] = airplane_df['classes'].apply(json.dumps)
        write_to_postgress(airplane_df,'airplanes')
        
        
            
    
    #Schedule flights
    def schedule_flights(self):
        while self.start_date <= self.end_date:
            
            curr_weekday = self.start_date.weekday()+1
            #print(curr_weekday)
            
            #trackers are used to remove and add airplanes from source to destination airport
            destination_tracker = { airport_name : set() for airport_name in self.airports}
            source_tracker = { airport_name : set() for airport_name in self.airports}

            
            flag = True
            
            #Will go through all airports and schedule flight for each airplane present in the airport
            for airport in self.airports:

                source_airport = airport          
                source_airport_data = self.airport_data[source_airport] 
                source_airplanes = source_airport_data.airplanes

                #running for each airplane
                
                
                
                for airplane in source_airplanes:

                    #print(airplane.name)
                    freq = self.airplane_initials[airplane.name]['frequency']
                    
                    size = len(self.airplane_initials[airplane.name]['timings'])
                    index = self.airplane_initials[airplane.name]['index']
                    
                    if(str(curr_weekday) in freq and index < size):
                        flag = False
                        
                        hour = self.airplane_initials[airplane.name]['hours']
                        time = self.airplane_initials[airplane.name]['timings'][index]
                        
                        #finding destination
                        destination_airport = self.airplane_initials[airplane.name]['destination']
                        print(f"{airplane.name} => {self.airplane_initials[airplane.name]} => {self.airplane_initials[airplane.name]['destination']}")
                        #scheduling_flight
                        flight = Flight(airplane,source_airport,destination_airport,self.global_flight_id,(self.start_date + timedelta(hours = time)),hour)
                        flight.fill_flight()
                        self.flights_data.append(flight.get_flight_data())

                        self.airplane_initials[airplane.name]['index'] += 1

                        #managing trackers
                        #print(destination_airport)
                        source_tracker[source_airport].add(airplane)
                        destination_tracker[destination_airport].add(airplane)

                        self.airplane_initials[airplane.name]['destination'] = self.airplane_initials[airplane.name]['source']
                        self.airplane_initials[airplane.name]['source'] = destination_airport

                        self.global_flight_id += 1

                        df = pd.DataFrame(self.flights_data)
                        df['departure_time'] = df['departure_time'].astype(str)
                        df['arrival_time'] = df['arrival_time'].astype(str)
                        df['passenger_data'] = df['passenger_data'].apply(json.dumps)
                        
                        #table = pa.Table.from_pandas(df)
                        #parquet_file_path = f'./data/Zathura/Year={str(self.start_date).split("-")[0]}/Month={str(self.start_date).split("-")[1]}/Day={str(self.start_date).split("-")[2][:2]}'
                        #os.makedirs(parquet_file_path, exist_ok=True)
                        #parquet_file_path = os.path.join(parquet_file_path, f'{str(self.start_date)[:10]}.parquet')
                        write_to_postgress(df,'flight_schedules')
                        self.flights_data = []
                        

                        
            #fixing trackers for the next run
            for airport in self.airports:
                self.airport_data[airport].airplanes = self.airport_data[airport].airplanes | destination_tracker[airport]
                self.airport_data[airport].airplanes = self.airport_data[airport].airplanes - source_tracker[airport]
                
            if(flag): 
                # df = pd.DataFrame(self.flights_data)
                # df['departure_time'] = df['departure_time'].astype(str)
                # df['arrival_time'] = df['arrival_time'].astype(str)
                # df['passenger_data'] = df['passenger_data'].apply(json.dumps)
                
                # #table = pa.Table.from_pandas(df)
                # #parquet_file_path = f'./data/Zathura/Year={str(self.start_date).split("-")[0]}/Month={str(self.start_date).split("-")[1]}/Day={str(self.start_date).split("-")[2][:2]}'
                # #os.makedirs(parquet_file_path, exist_ok=True)
                # #parquet_file_path = os.path.join(parquet_file_path, f'{str(self.start_date)[:10]}.parquet')
                # self.flights_data = []
                # write_to_postgress(df)

                # Write the PyArrow Table to Parquet file
                #pq.write_table(table, parquet_file_path)
                
                self.start_date = self.start_date + timedelta(days = 1)
                for airplane in self.airplane_data:
                    self.airplane_initials[airplane.name]['index'] = 0
                
                

    def get_print(self):
        for airport in self.airports:
            curr_airport = self.airport_data[airport]
            print(f"{curr_airport.name}- ")
            for ap in curr_airport.airplanes:
                print(f" \t\t {ap.name}")
                
                
    def show_data(self):
        flight_schedules_df = pd.DataFrame(self.flights_data)
        display(flight_schedules_df)
        
        

In [71]:
zathura = Zathura()
zathura.set_airplane_data()
zathura.set_airport_data()
zathura.set_inital_airplanes_to_airports()
#zathura.load_initaltables()
zathura.get_print()

Mumbai- 
 		 Vimana
 		 Marut
 		 Rudra
Delhi- 
 		 Ananta
Pune- 
 		 Garuda
 		 Vajra
Banglore- 
 		 Astra
 		 Agni
Hydrebad- 
 		 Vayu


In [ ]:
zathura.schedule_flights()

Vimana => {'frequency': '1357', 'source': 'Mumbai', 'destination': 'Delhi', 'timings': [6, 12, 18], 'index': 0, 'hours': 2} => Delhi
Marut => {'frequency': '1234567', 'source': 'Mumbai', 'destination': 'Pune', 'timings': [6, 12, 18], 'index': 0, 'hours': 1} => Pune
Rudra => {'frequency': '1234567', 'source': 'Mumbai', 'destination': 'Hydrebad', 'timings': [12, 17], 'index': 0, 'hours': 2} => Hydrebad
